In [2]:
import pandas as pd

df=pd.read_csv("health_data.csv")
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   heart_rate      500 non-null    int64
 1   activity_level  500 non-null    int64
 2   age             500 non-null    int64
 3   risk_level      500 non-null    int64
dtypes: int64(4)
memory usage: 15.8 KB


,heart_rate,activity_level,age,risk_level
count,500.000000,500.000000,500.000000,500.000000
mean,79.180000,5.240000,45.022000,0.464000
std,11.718628,2.449162,14.661495,0.499202
min,60.000000,1.000000,20.000000,0.000000
25%,69.000000,3.000000,32.000000,0.000000
50%,79.000000,5.000000,46.000000,0.000000
75%,89.000000,7.000000,58.000000,1.000000
max,99.000000,9.000000,69.000000,1.000000


In [3]:
from sklearn.model_selection import train_test_split
features = ["heart_rate","activity_level","age"]
X = df[features]
label = "risk_level"
y= df[label]
test_size = 0.2
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size,random_state = 42)

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100,random_state = 42 )
rf.fit(X_train,y_train)


RandomForestClassifier(random_state=42)

In [5]:
def predict_health_risks(heart_rate,activity_level,age):
    input_data = pd.DataFrame({"heart_rate":[heart_rate],"activity_level":[activity_level],"age":[age]})
    prediction = rf.predict(input_data)[0]
    risk = "high" if prediction == 1 else "low"    
    return risk

In [11]:
!pip install openai
from openai import OpenAI



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
client = OpenAI(
    api_key=""
)

In [16]:
def provide_health_advice(heart_rate, activity_level, age):
    risk = predict_health_risks(heart_rate, activity_level, age)
    completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a health advisor."},
            {"role": "user", "content": f"My heart rate is {heart_rate}, my activity level is {activity_level}, and I am {age} years old. My risk level is {risk}. Provide personalized health advice."}
        ],
        model="gpt-3.5-turbo",
    )
    advice = completion.choices[0].message.content.strip()
    return advice

In [18]:
for index, row in df.iterrows():
    heart_rate = row['heart_rate']
    activity_level = row['activity_level']
    age = row['age']
 
    risk = predict_health_risks(heart_rate, activity_level, age)
    print(f"Predicted Health Risk for User {index}: {risk}")
 
    advice = provide_health_advice(heart_rate, activity_level, age)
    print(f"\nHealth Advice for User {index}:\n{advice}")

Predicted Health Risk for User 0: low


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}